In [ ]:
# Text to cube/cif/xsf
import numpy as np
import math as ma
from pathlib import Path
import ase.io
import ase
import ase.build
from text_to_gridfiles import grid_text_to_cube


unitcell = ase.io.read("Melania_data/init_cif/54189.cif")
energyfile = "test_energies_54189.txt"
coordfile = "test_coords_54189.txt"

grid_shape = (2,2,2)


grid_text_to_cube(energyfile, coordfile, unitcell, grid_shape, name="TESTCUBEOUT")

In [ ]:
atoms = ase.io.read("Melania_data/init_cif/54189.cif")
a ,b, c = atoms.get_cell()[0], atoms.get_cell()[1], atoms.get_cell()[2]

a_mesh = np.linspace(0, a, grid_shape[0], endpoint=False)
b_mesh = np.linspace(0, b, grid_shape[1], endpoint=False)
c_mesh = np.linspace(0, c, grid_shape[2], endpoint=False)

coords = []
for i in a_mesh:
    for j in b_mesh:
        for k in c_mesh:
            coords.append(i + j + k)

print(np.array(coords))

np.savetxt("test_coords_54189.txt", np.array(coords))


In [ ]:
test_E = np.loadtxt('test_energies_54189.txt')

In [ ]:
test_E


In [ ]:
import numpy as np
a,b,c = np.arange(0,3),np.arange(3,7),np.arange(7,10)
A = np.meshgrid(a,b,c,indexing='xy')

np.array(A)[1,1,1]

np.ravel_multi_index(([2,3,4],[4,6,3], [1,2,3]),dims=(10,10,10))

#print(4*100+6*10+3)

In [1]:
import itertools as it
import numpy as np
import math as ma
from pathlib import Path
import ase.io
import ase
import ase.build
from text_to_gridfiles import grid_text_to_cube

unitcell = ase.io.read("Melania_data/init_cif/54189.cif")
energyfile = "test_half_energies_54189.txt"
coordfile = "test_half_coords_54189.txt"


grid_values = np.loadtxt(energyfile)
coords_computed = np.loadtxt(coordfile)
grid_shape = [2,2,2]

# Debug:
print("grid_values shape: ", grid_values.shape)
print("coords_computed shape: ", coords_computed.shape)
#

fill_value = 999.0

a, b, c = unitcell.get_cell()[:]

grid = np.empty((np.product(grid_shape)), dtype=np.float64)
f_coords = np.array(list(it.product(np.linspace(0, 1, grid_shape[0], endpoint=False), np.linspace(0, 1, grid_shape[1], endpoint=False), np.linspace(0, 1, grid_shape[2], endpoint=False)))) 
coords1 = f_coords @ unitcell.get_cell()
#coords2 = np.array(list(it.product(np.linspace(0, a, grid_shape[0], endpoint=False), np.linspace(0, b, grid_shape[1], endpoint=False), np.linspace(0, c, grid_shape[2], endpoint=False))))
coords2 = []
for i in np.linspace(0, a, grid_shape[0], endpoint=False):
    for j in np.linspace(0, b, grid_shape[1], endpoint=False):
        for k in np.linspace(0, c, grid_shape[2], endpoint=False):
            coords2.append(i+j+k)

coords2 = np.array(coords2)


print(coords1)
print(coords2)
assert (coords1 == coords2).all(), "Coords 1 and 2 are different!!!"


print("grid shape: ", grid.shape)
print("coords shape: ", coords1.shape)


grid.fill(fill_value)

# Compute indices from coordinates

indices = np.rint(np.linalg.solve(unitcell.get_cell().T , coords_computed.T).T * np.array(grid_shape)).astype(dtype=int)
print(indices)

raveled_indices  = np.ravel_multi_index((indices[:,0], indices[:,1], indices[:,2]), grid_shape)
print(raveled_indices)

print("Coords raveled indices: ", (np.array(coords1)[raveled_indices]))
print("Coords computed: ", np.array(coords_computed))

assert (np.array(coords1)[raveled_indices] == np.array(coords_computed)).all(), "coords are incorrect."

grid = grid.flatten()
grid[raveled_indices] = grid_values

print(grid)

# Debug:
#print(indices)
#x_ind, y_ind, z_ind = indices[:, 0], indices[:, 1], indices[:, 2]
#grid[x_ind, y_ind, z_ind] = grid_values
#coords[x_ind, y_ind, z_ind, :] = coords_computed"""


test_grid, test_coords, test_indices, test_raveled_indices = grid_text_to_cube(energyfile,
                                                        coordfile,
                                                        unitcell,
                                                        grid_shape,
                                                        fill_value=fill_value,
                                                        write_file=False,
                                                        return_grids=True)

assert (test_grid == grid).all(), "Grids are not the same!!!"
assert (test_coords == coords1).all(), "Coords are not the same!!!"
assert (test_indices == indices).all(), "Indices are not the same!!!"
assert (test_raveled_indices == raveled_indices).all(), "Raveled indicies are not the same!!!"


print("test_grid: ", test_grid)

if (test_grid == grid).all() and (test_coords == coords1).all() and (test_indices == indices).all() and (test_raveled_indices == raveled_indices).all():
    print("Everything is the same. Function test successful.")
else: 
    print("Something differs between the function output and the script!!! Test failed.")



grid_values shape:  (4,)
coords_computed shape:  (4, 3)
[[0.         0.         0.        ]
 [1.08982068 0.62920826 1.77966971]
 [1.08982068 1.88762478 0.        ]
 [2.17964135 2.51683304 1.77966971]
 [2.17964135 0.         0.        ]
 [3.26946203 0.62920826 1.77966971]
 [3.26946203 1.88762478 0.        ]
 [4.3592827  2.51683304 1.77966971]]
[[0.         0.         0.        ]
 [1.08982068 0.62920826 1.77966971]
 [1.08982068 1.88762478 0.        ]
 [2.17964135 2.51683304 1.77966971]
 [2.17964135 0.         0.        ]
 [3.26946203 0.62920826 1.77966971]
 [3.26946203 1.88762478 0.        ]
 [4.3592827  2.51683304 1.77966971]]
grid shape:  (8,)
coords shape:  (8, 3)
[[0 0 0]
 [0 1 0]
 [1 0 0]
 [1 1 0]]
[0 2 4 6]
Coords raveled indices:  [[0.         0.         0.        ]
 [1.08982068 1.88762478 0.        ]
 [2.17964135 0.         0.        ]
 [3.26946203 1.88762478 0.        ]]
Coords computed:  [[0.         0.         0.        ]
 [1.08982068 1.88762478 0.        ]
 [2.17964135 0.    